This notebook takes the skill action data and enrchies it with the item mapping data, specifically to get the item unique ID's
The ID is easier to work with in future joins

In [0]:
import pyspark.sql.functions as sf

In [0]:
# Read data from bronze layer Unity Catalog table 'runescape.01_bronze.skill_action_xp'
df_skill_xp = spark.read.table("runescape.01_bronze.skill_action_xp")

In [0]:
# Read data from bronze layer Unity Catalog table 'runescape.01_bronze.skill_action_xp'
df_skill_inputs = spark.read.table("runescape.01_bronze.skill_action_inputs")

In [0]:
# Read data from silver layer mapping table 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping").\
    select("name", "id")

In [0]:
# Join the dataframes to create enriched dataframe
df_skill_xp_enriched = df_skill_xp.join(df_map, df_map.name == df_skill_xp.output)
df_skill_xp_enriched = df_skill_xp_enriched.select("output", "xp", "skill", "id")

In [0]:
# Join the dataframes to create enriched dataframe
# Join data frames to get output ID
df_skill_inputs_enriched = df_skill_inputs.join(df_map, df_map.name == df_skill_inputs.output)
df_skill_inputs_enriched = df_skill_inputs_enriched.withColumnRenamed("id", "id_output")
# Join data frames to get input ID
df_skill_inputs_enriched = df_skill_inputs_enriched.join(df_map, df_map.name == df_skill_inputs.input)
df_skill_inputs_enriched = df_skill_inputs_enriched.withColumnRenamed("id", "id_input")
# df_skill_inputs_enriched
df_skill_inputs_enriched = df_skill_inputs_enriched.select("output", "input_qty", "input", "skill", "id_output", "id_input")

In [0]:
df_skill_inputs_enriched.display()

In [0]:
# save data to bronze layer Unity Catalog table 'runescape.02_silver.skill_action_xp_enriched'
df_skill_xp_enriched.write.mode("overwrite").saveAsTable("runescape.02_silver.skill_action_xp_enriched")

In [0]:
# save data to bronze layer Unity Catalog table 'runescape.02_silver.skill_action_inputsenriched'
df_skill_inputs_enriched.write.mode("overwrite").saveAsTable("runescape.02_silver.skill_action_inputs_enriched")